In [ ]:
# DO NOT MODIFY THIS CELL - this cell is splitting the data to provide a suitable subset of data to work with for this task.
# If you change this cell your output will differ from that expected and could impact your mark.
import random
import string
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
#import datetime
from statistics import mean
from datetime import datetime
import re
import numpy as np
import os

dataset_list = ['onlinebargains']
DATASET = dataset_list[0]
def load_data(DATASET):
    if DATASET in dataset_list:
        email_data = pd.read_csv('./T3_data/' + DATASET + '/email_data.csv', parse_dates=True, index_col=0)
        file_data = pd.read_csv('./T3_data/' + DATASET + '/file_data.csv', parse_dates=True, index_col=0)
        web_data = pd.read_csv('./T3_data/' + DATASET + '/web_data.csv', parse_dates=True, index_col=0)
        login_data = pd.read_csv('./T3_data/' + DATASET + '/login_data.csv', parse_dates=True, index_col=0)
        usb_data = pd.read_csv('./T3_data/' + DATASET + '/usb_data.csv', parse_dates=True, index_col=0)
        employee_data = pd.read_csv('./T3_data/' + DATASET + '/employee_data.csv', index_col=0)
        
        email_data['datetime'] = pd.to_datetime(email_data['datetime'])
        file_data['datetime'] = pd.to_datetime(file_data['datetime'])
        web_data['datetime'] = pd.to_datetime(web_data['datetime'])
        login_data['datetime'] = pd.to_datetime(login_data['datetime'])
        usb_data['datetime'] = pd.to_datetime(usb_data['datetime'])
    else:
        print ("DATASET variable not defined")
        return
    return employee_data, login_data, usb_data, web_data, file_data, email_data

employee_data, login_data, usb_data, web_data, file_data, email_data = load_data(DATASET)
def check_data(data,operation):
    suspecious=[]
    logoff=check_login(login_data)
    for i in range(1,12):
        month=str(i)
        if i< 10 :
            month='0'+str(i)
        logoff=logoff[logoff['month']==month]
        print('Login for Month ('+str(month)+'): '+str(len(logoff)))
        print('Kindly Wait.. Extracting data')
        datalist = pd.DataFrame({"user": data['user'].tolist(),           # Create pandas DataFrame
                            "date": data['datetime'].dt.date,
                            "time": data['datetime'].dt.strftime("%H:%M:%S"),
                            "datetime":  data['datetime'].tolist(),
                            "month": data["datetime"].dt.strftime('%m')})
        logoff_data=np.array(logoff)
        datalist=datalist[datalist['month']== month]
        print('Length of '+operation+' of Month ('+str(month)+'): '+str(len(datalist)))
        print('Kindly Wait.. Comparing data')
        for row in logoff_data:       
                data_new=datalist[(datalist['user']== row[0]) & (pd.to_datetime(datalist['date'])==  pd.to_datetime(row[1]))] #   'user' & logoff_data['date'] == item[1]
                max_date=np.max(pd.to_datetime(data_new['datetime']))
                maxRow=data_new[data_new['datetime']==max_date]
                data_arr=np.array(maxRow)
                for item in data_arr:
                    if(pd.to_datetime(item[3]) >= pd.to_datetime(row[4])):
                        suspecious.append(row)
        if len(suspecious)>0:                
            print("Suspecious users for Month ("+month+") are:")        
            [print(p[0]) for p in suspecious]
        else:
            print("No Suspecious found on month ("+month+")")

def check_login(login_data):
    subset = pd.DataFrame({"user": login_data['user'].tolist(),
                          "date": login_data['datetime'].dt.date,
                          "time": login_data['datetime'].dt.strftime("%H:%M:%S"),
                          "action": login_data['action'].tolist(),
                          "datetime":  login_data['datetime'].tolist(),
                          "month": login_data["datetime"].dt.strftime('%m')})
    logoff_data=subset[subset["action"]=="logoff"]
    return logoff_data
def check_maximum(data,operation,user,isInvest):
    print("")
    print("-------------------------------")
    print("Start "+operation+" Investigation")
    print("---------------------------------")
    print("Please Wait...")
    print("")
    data["month"] = data["datetime"].dt.strftime('%m')
    subset = pd.DataFrame({"user": data[user].tolist(),           # Create pandas DataFrame
                          "month": data['month'].tolist(),
                          "summation": 0})
    df_data = subset.groupby(['user','month'],as_index=False).agg({'summation':'count'})
    if(isInvest==False):
        print("Distinct Data")
        print("-------------")
        print(df_data)

    distinct_data = df_data['user'].tolist()
    distinct_data = list(set(distinct_data))
    avgList=[]
    maxList=[]
    maximum_no=0
    maxAverage_no=0
    maximum_data=""
    maxAverage_data=""
    
    for i in distinct_data:
       data_month=df_data[df_data['user'] == i]
       meanValue=np.floor(mean(data_month['summation']))
       maxValue=np.max(data_month['summation'])
       maxRow=np.array(data_month[data_month['summation']==maxValue])
       maxMonth=maxRow[0,1]    
       
       maxList.append([i,maxMonth, maxValue])
       avgList.append([i, meanValue])
       plt.plot(data_month.month,data_month.summation)
       if maxValue > maximum_no :
          maximum_no=maxValue
          maximum_data=maxRow[0,0]
       
       if meanValue > maxAverage_no :
          maxAverage_no=meanValue
          maxAverage_data=maxRow[0,0]

    maxList_new = pd.DataFrame(maxList, columns=['User', 'Month', 'Max Value'])
    avgList_new = pd.DataFrame(avgList, columns=['User', 'Average Value'])
    if(isInvest==False):
        print("Maximum Months Per User")
        print("-----------------------")
        print(maxList_new)
        print("")
        print("Average Per User")
        print("-----------------------")
        print(avgList_new)
        print("")
        plt.show()
    print("Suspecious Users of "+operation) 
    print("-------------------------------")   
    print("1- Maximum value="+str(maximum_no))
    print("   User Name: "+str(maximum_data))
    print("2- Maxmimum Average value="+ str(maxAverage_no))
    print("   User Name: "+str(maxAverage_data))
    print("")

def display_menu():
    print("1 - Email Investigation")
    print("2 - File Investigation")
    print("3 - USB Investigation")
    print("4 - Web Investigation")
    print("5 - All Investigation")
    print("Q - Quit")
def main():   
    display_menu()  
    choice = input("Enter your choice: ")
    while choice != "Q" and choice != "q":
        os.system("cls")    
        if choice=="1":
            check_maximum(email_data,"Email Sent",'sender',False) 
        elif choice=="2":
            check_maximum(file_data,"File Access",'user',False)
            #check_data(file_data,"File Access") #check accessed files
        elif choice=="3":
            check_maximum(usb_data,"USB Usage",'user',False)
            #check_data(usb_data,"USB Usage")
        elif choice=="4":
            check_maximum(web_data,"Web Access",'user',False)
            #check_data(web_data,"Web Access")
        elif choice=="5":
            check_maximum(email_data,"Email Sent",'sender',True)
            check_maximum(file_data,"File Access",'user',True)
            check_maximum(usb_data,"USB Usage",'user',True)
            check_maximum(web_data,"Web Access",'user',True)
          
        else:
            print("Invalid Choice.")
        display_menu()
        choice = input("Enter your choice: ")

if __name__ == "__main__":
    main()